In [51]:
# Load main dataset (new_raw_data_polygon.csv) and merge with cluster labels
main_data_path = '../data/raw_data/new_raw_data_polygon.csv'
main_df = pd.read_csv(main_data_path)
main_cluster_labels = cluster_labels['main']
if 'cluster_label' not in main_cluster_labels.columns:
    raise ValueError('cluster_label column not found in cluster labels for main dataset')
if 'id' in main_df.columns and 'id' in main_cluster_labels.columns:
    merged_main = pd.merge(main_df, main_cluster_labels, on='id', how='left')
else:
    # If no common id, merge by index (assumes order is preserved)
    merged_main = main_df.copy()
    merged_main['cluster_label'] = main_cluster_labels['cluster_label']
print('Merged main dataset shape:', merged_main.shape)
merged_main.head()

Merged main dataset shape: (20174, 23)


,WALLET,TX_PER_MONTH,TOKEN_DIVERSITY,PROTOCOL_DIVERSITY,TOTAL_TRANSFER_USD,INTERACTION_DIVERSITY,ACTIVE_DURATION_DAYS,AVG_TRANSFER_USD,USD_TRANSFER_STDDEV,DEX_EVENTS,...,CHADMIN_EVENTS,DEFI_EVENTS,BRIDGE_EVENTS,NFT_EVENTS,TOKEN_EVENTS,FLOTSAM_EVENTS,BRIDGE_OUTFLOW_COUNT,BRIDGE_INFLOW_COUNT,BRIDGE_TOTAL_VOLUME_USD,cluster_label
0,0xecb113be97a619aa4d070491505847000f964448,8.166667,6,9,2840.47,5,177,41.166232,88.745481,5,...,0,0,1,0,28,0,1,0,499.88,9
1,0x76c1cf35f54c67bc4fec5ba1411f5c0825a4a50d,8.666667,2,8,3733.59,10,172,56.569545,185.180716,1,...,0,0,0,0,0,0,0,0,0.00,-1
2,0x2465ba1d82c7faf5cfc1b4f0e3c606800e1caac9,13.000000,5,26,3771.93,14,169,62.865500,135.890912,0,...,0,0,6,0,23,0,0,0,0.00,15
3,0xa5a1a2abcc2b67990b37b749c4e840f1fa093c97,6.833333,1,2,12977.28,2,170,370.779429,318.130714,0,...,0,0,0,0,35,0,0,0,0.00,5
4,0xa967e15e71455940dfab4c08719abae50401babd,7.500000,5,6,18935.76,6,156,236.697000,478.737614,5,...,0,0,10,0,25,0,9,0,1206.05,23


In [ ]:
feature_analysis_results = {}
for cluster in sorted(merged_main['cluster_label'].dropna().unique()):
    cluster_df = merged_main[merged_main['cluster_label'] == cluster]
    # Use the feature analysis function from interaction_mode_features_v2.py
    # Replace 'your_feature_analysis_function' with the actual function name
    # Example: result = interaction_mode_features_v2.analyze_features(cluster_df)
    # For demonstration, we'll just compute means for all numeric columns
    result = cluster_df.describe().T[['mean', 'std', 'min', 'max']]
    feature_analysis_results[cluster] = result
# Combine all results into a single DataFrame for easy comparison
combined_feature_analysis = pd.concat(feature_analysis_results, names=['cluster_label'])
combined_feature_analysis

mean           std    min  \
cluster_label                                                               
-1            TX_PER_MONTH                 20.673600  5.276391e+01   1.00   
              TOKEN_DIVERSITY               4.328756  2.572299e+00   1.00   
              PROTOCOL_DIVERSITY           11.151454  1.558106e+01   1.00   
              TOTAL_TRANSFER_USD       119759.468013  3.557831e+06   0.03   
              INTERACTION_DIVERSITY         9.418417  3.833743e+00   1.00   
...                                              ...           ...    ...   
 24           FLOTSAM_EVENTS                0.000000  0.000000e+00   0.00   
              BRIDGE_OUTFLOW_COUNT        101.991905  2.125126e+03   0.00   
              BRIDGE_INFLOW_COUNT           0.000000  0.000000e+00   0.00   
              BRIDGE_TOTAL_VOLUME_USD  120443.300143  1.994746e+06   0.00   
              cluster_label                24.000000  0.000000e+00  24.00   

                                                max  
cluster_label                                        
-1            TX_PER_MONTH             2.221000e+03  
              TOKEN_DIVERSITY          3.300000e+01  
              PROTOCOL_DIVERSITY       6.750000e+02  
              TOTAL_TRANSFER_USD       1.671072e+08  
              INTERACTION_DIVERSITY    3.900000e+01  
...                                             ...  
 24           FLOTSAM_EVENTS           0.000000e+00  
              BRIDGE_OUTFLOW_COUNT     7.523600e+04  
              BRIDGE_INFLOW_COUNT      0.000000e+00  
              BRIDGE_TOTAL_VOLUME_USD  6.795146e+07  
              cluster_label            2.400000e+01  

[572 rows x 4 columns]